## LLM-as-Judge

In [ ]:
# ===== All Required Libraries =====

# Environment Setup
from dotenv import load_dotenv

# LangChain Related
from langchain_teddynote import logging
from langchain_openai import ChatOpenAI

# Data Processing
import pandas as pd

# LangSmith
from langsmith import Client

# Custom Modules
from RAGPDF import PDFRAG
from evaluator import correctness, groundedness, helpfulness, retrieval_relevance

In [ ]:
# Load API KEY
load_dotenv()

True

In [ ]:
from langchain_teddynote import logging

# Put project name
logging.langsmith("Evaluation Test")

## Load Synthetic dataset

In [ ]:
df = pd.read_csv("../ragas_synthetic_dataset.csv")

In [ ]:
df.head()

### Set RAG

In [ ]:
FILE_PATH = "Your file path"

In [ ]:
from RAGPDF import PDFRAG
from langchain_openai import ChatOpenAI

rag = PDFRAG(file_path=FILE_PATH, llm=ChatOpenAI(model="gpt-4.1-mini", temperature=0))

# Creat Retriever
retriever = rag.create_retriever()

# Create RAG chain
chain = rag.create_chain(retriever)

# generate answer
chain.invoke("2024년 UN e-Government Survey에서 호주 순위를 알려줘")

In [7]:
def ask_question(inputs: dict) -> dict:
    # extract value of "input"
    question = inputs.get("input")

    # retrieve documents
    retrieved_docs = retriever.invoke(question)
    answer = chain.invoke(question)

    return {"answer": answer, "documents": retrieved_docs}

In [8]:
## test code for RAG

# generated_answer = ask_question(

#     {"input": "2024년 UN e-Government Survey에서 호주 순위를 알려줘"}

# )

# print(generated_answer)

## Create Dataset for LangSmith Test 

In [9]:
## Use this function to create a dataset in LangSmith

# def create_dataset(client, dataset_name, description=None):
#     for dataset in client.list_datasets():
#         if dataset.name == dataset_name:
#             return dataset

#     dataset = client.create_dataset(
#         dataset_name=dataset_name,
#         description=description,
#     )

#     return dataset

In [ ]:
client = Client()
dataset_name = "RAG_EVAL_DATASET"

In [11]:
### Use this code add examples to dataset in LangSmith

### Create Dataset

# dataset = create_dataset(client, dataset_name)


### add examples to dataset

# client.create_examples(

#     inputs=[{"input": q} for q in df["user_input"]],

#     outputs=[{"output": q} for q in df["reference"]],

#     dataset_id=dataset.id,

# )

## RUN Evaluation with LangSmith

In [12]:
from evaluator import correctness, groundedness, helpfulness, retrieval_relevance

metrics = [correctness, groundedness, helpfulness, retrieval_relevance]

In [13]:
dataset_name = "RAG_EVAL_DATASET"

# run evaluation
experiment_results = client.evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[correctness, groundedness, helpfulness, retrieval_relevance],
    experiment_prefix="rag-test_eval",
    metadata={"model": "gpt-4.1-mini", "variant": "RAG EVALUATION"},
)

c:\Users\jongb\dev\jb_langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-test_eval-88133be1' at:
https://smith.langchain.com/o/d509ef49-7de3-442d-8ac5-641ba83c50aa/datasets/a3485372-bd84-42da-ac46-c5776061f0cb/compare?selectedSessions=18f33eed-267c-4f11-b3db-db8186adaea4




12it [04:28, 22.37s/it]
